# Deploying an AML Model as a UDF using MLFlow

In [0]:
import mlflow
from azureml.core import Workspace, Model

ws = Workspace(workspace_name="ws-aml-dbx-demo", resource_group="rg-aml-dbx-demo", subscription_id="02559c12-8982-43b3-8180-f4e41779489b")
model = Model(ws, name="diabetes-xgboost", version=1)

# Download the model to the DBFS
model_path = model.download("dbfs:/FileStore/models/diabetes/xgb", exist_ok=True)

In [0]:
# Note - the diabetes.csv has been uploaded to DBFS here and is being loaded in here...
df1 = spark.read.format("csv").option("header", True).option("inferSchema", True).load("dbfs:/FileStore/shared_uploads/diabetes/diabetes.csv")

# Create UDF and apply it to the Dataframe
xgb_udf = mlflow.pyfunc.spark_udf(spark, model_path)
df1 = df1.withColumn("prediction", xgb_udf("age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"))

display(df1)